### A/B Hypothesis Testing
    Accept or reject the following Null Hypotheses: 
        There are no risk differences across provinces 
        There are no risk differences between zip codes 
        There are no significant margin (profit) difference between zip codes 
        There are not significant risk difference between Women and Men


### There are no risk differences across provinces 

In [1]:
import os 
os.chdir("../")

In [2]:
import pandas as pd
df = pd.read_csv("data/dataset.csv")

C:\Users\windows 10\AppData\Local\Temp\ipykernel_648\685072172.py:2: DtypeWarning: Columns (33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/dataset.csv")


In [3]:
from scripts.preprocessing import replace_missing_with_mean, replace_missing_with_mode

df = df.drop(columns=['NumberOfVehiclesInFleet','CrossBorder' ])
numerical_cols = df.select_dtypes(include=['int64', 'float64'])
categorical_cols = df.select_dtypes(include='object')

df = replace_missing_with_mode(df,categorical_cols)
df = replace_missing_with_mean(df,numerical_cols.columns)


In [5]:
# A/B Hypothesis Testing methods

import pandas as pd
from scipy.stats import chi2_contingency


def AB_haypothe(df,col):
    # Calculate loss ratio
    df["LossRatio"] = df["TotalClaims"] / df["SumInsured"]

    # Define high-risk threshold
    high_risk_threshold = 1

    # Create high-risk indicator
    df["HighRisk"] = df["LossRatio"] > high_risk_threshold

    # Create contingency table
    contingency_table = pd.crosstab(df[col], df["HighRisk"])

    # Perform chi-squared test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)

    return chi2, p_value, dof, expected


def calculate_profit_margin_and_anova(df):
   

    # Calculate profit margin
    df['ProfitMargin'] = (df['TotalPremiums'] - df['TotalClaims']) / df['TotalPremiums']

    # Group df by zip code and calculate average profit margin
    grouped_df = df.groupby("PostalCode")["ProfitMargin"].mean()

    # Perform ANOVA
    f_statistic, p_value = f_oneway(*grouped_df)

    return f_statistic, p_value




def calculate_loss_ratio_and_t_test_for_gender(data, gender_column, loss_ratio_column):
    
    # Calculate loss ratio if it doesn't exist
    if loss_ratio_column not in data.columns:
        data[loss_ratio_column] = data['TotalClaims'] / data['SumInsured']

    # Group data by gender
    female_data = data[data[gender_column] == "Female"][loss_ratio_column]
    male_data = data[data[gender_column] == "Male"][loss_ratio_column]

    # Perform t-test
    t_statistic, p_value = ttest_ind(female_data, male_data)

    return t_statistic, p_value







In [6]:
from scripts.Hypothesis_Testing import AB_haypothe
chi2, p_value, dof, expected = AB_haypothe(df,"Province")
if p_value < 0.05:
    print("Reject null hypothesis: There are significant risk differences across provinces.")
else:
    print("Fail to reject null hypothesis: There are no significant risk differences across provinces.")

NameError: name 'ab_hypothesis' is not defined

In [ ]:
if p_value < 0.05:
    print("Reject null hypothesis: There are significant risk differences across provinces.")
else:
    print("Fail to reject null hypothesis: There are no significant risk differences across provinces.")

### There are no risk differences between zip codes 


In [ ]:

chi2_zip, p_value_zip, dof_zip, expected_zip = ab_hypothesis(df,"PostalCode")

# Interpret results for zip codes
if p_value_zip < 0.05:
    print("Reject null hypothesis: There are significant risk differences between zip codes.")
else:
    print("Fail to reject null hypothesis: There are no significant risk differences between zip codes.")

### There are no significant margin (profit) difference between zip codes 


In [ ]:
from scripts.Hypothesis_Testing import calculate_profit_margin_and_anova


f_statistic, p_value = calculate_profit_margin_and_anova(df)

# Interpret results
if p_value < 0.05:
    print("Reject null hypothesis: There are significant profit margin differences between zip codes.")
else:
    print("Fail to reject null hypothesis: There are no significant profit margin differences between zip codes.")

### There are not significant risk difference between Women and Men


In [ ]:
from scripts.Hypothesis_Testing import calculate_loss_ratio_and_t_test_for_gender

t_stat, p_value = calculate_loss_ratio_and_t_test_for_gender(df, "Gender", "LossRatio")


# Interpret results
if p_value < 0.05:
    print("Reject null hypothesis: There are significant risk differences between women and men.")
else:
    print("Fail to reject null hypothesis: There are no significant risk differences between women and men.")